In [1]:
import os
import sys
import tensorflow as tf
import numpy as np
import pathlib
import urllib.request
from importlib import reload
from magenta.models.melody_rnn import melody_rnn_model
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from magenta.models.rl_tuner import rl_tuner
from magenta.models.rl_tuner import rl_tuner_ops
from note_seq import midi_io
from note_seq.protobuf import music_pb2, generator_pb2
import note_seq
import glob


c:\Users\adamc\miniconda3\envs\magenta_legacy\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\adamc\miniconda3\envs\magenta_legacy\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\adamc\miniconda3\envs\magenta_legacy\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
 

In [2]:
# Disable TF v2 behavior for compatibility
tf.compat.v1.disable_v2_behavior()

# Configuration
CONFIG = 'basic_rnn'  #Note_RNN
MELODY_RNN_SAVE_PATH = "./melody_rnn_finetuned/"

# Create save directory
os.makedirs(MELODY_RNN_SAVE_PATH, exist_ok=True)

# Download and load pre-trained Melody_RNN model
model_cache_dir = pathlib.Path.home() / '.magenta' / 'models'
model_cache_dir.mkdir(parents=True, exist_ok=True)
bundle_file = model_cache_dir / f'{CONFIG}.mag'

print(f"Loading pre-trained {CONFIG} model...")

if not bundle_file.exists():
    print(f"  Downloading {CONFIG} model...")
    bundle_url = f'http://download.magenta.tensorflow.org/models/{CONFIG}.mag'
    try:
        urllib.request.urlretrieve(bundle_url, str(bundle_file))
        print(f"  ✓ Downloaded to {bundle_file}")
    except Exception as e:
        print(f"  Error: {e}")
        raise
else:
    print(f"  ✓ Using cached model: {bundle_file}")

# Load the bundle
config = melody_rnn_model.default_configs[CONFIG]
config.hparams.parse('')
bundle = sequence_generator_bundle.read_bundle_file(str(bundle_file))

Instructions for updating:
non-resource variables are not supported in the long term
Loading pre-trained basic_rnn model...
  ✓ Using cached model: C:\Users\adamc\.magenta\models\basic_rnn.mag


# Data Preprocessing & Fine-Tuning Pipeline

This section handles data preparation and model fine-tuning for artist-specific models.


In [ ]:
## Step 3: Fine-Tune Model on Artist-Specific Data

# Fine-tuning configuration
FINETUNE_CONFIG = {
    'learning_rate': 0.001,
    'batch_size': 32,
    'num_steps': 5000,
    'steps_per_checkpoint': 500,
}

def fine_tune_melody_rnn(artist_name, tfrecord_path, config, output_dir):
    """
    Fine-tune Melody_RNN using Magenta's training pipeline.
    
    Args:
        artist_name: name of the artist
        tfrecord_path: path to TFRecord training data
        config: training configuration dict
        output_dir: where to save fine-tuned model
    """
    import subprocess
    
    print(f"\n🎹 Fine-tuning for {artist_name}...")
    print(f"  Learning rate: {config['learning_rate']}")
    print(f"  Training steps: {config['num_steps']}")
    print(f"  Batch size: {config['batch_size']}")
    
    # Create output directory for this artist
    artist_model_dir = os.path.join(output_dir, artist_name)
    os.makedirs(artist_model_dir, exist_ok=True)
    train_dir = os.path.join(artist_model_dir, 'train')
    os.makedirs(train_dir, exist_ok=True)
    
    # Get bundle file path
    model_cache_dir = pathlib.Path.home() / '.magenta' / 'models'
    bundle_file = model_cache_dir / f'{CONFIG}.mag'
    
    print(f"\n  Starting training...\n")
    
    try:
        # Use Magenta's melody_rnn_train script
        cmd = [
            'melody_rnn_train',
            f'--config={CONFIG}',
            f'--bundle_file={str(bundle_file)}',
            f'--output_dir={train_dir}',
            f'--num_training_steps={config["num_steps"]}',
            f'--batch_size={config["batch_size"]}',
            f'--learning_rate={config["learning_rate"]}',
            f'--tfrecord_path={tfrecord_path}',
        ]
        
        # Run training
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            print(f"  ✓ Training completed successfully")
            final_checkpoint = os.path.join(train_dir, f'model.ckpt-{config["num_steps"]}')
        else:
            print(f"  ⚠ Training output:")
            print(result.stdout)
            print(result.stderr)
            final_checkpoint = os.path.join(train_dir, 'model.ckpt-0')
        
        print(f"  Final checkpoint: {final_checkpoint}")
        
        return {
            'artist': artist_name,
            'train_dir': train_dir,
            'final_checkpoint': final_checkpoint,
            'steps_trained': config['num_steps'],
        }
    
    except FileNotFoundError:
        print(f"  ⚠ melody_rnn_train command not found")
        print(f"  Install Magenta: pip install magenta")
        # Fallback: create checkpoint directory structure
        final_checkpoint = os.path.join(train_dir, 'model.ckpt-0')
        os.makedirs(train_dir, exist_ok=True)
        return {
            'artist': artist_name,
            'train_dir': train_dir,
            'final_checkpoint': final_checkpoint,
            'steps_trained': 0,
        }

# Fine-tune for selected artist
print("\n" + "="*60)
print("FINE-TUNING MODELS")
print("="*60)

ARTISTS_TO_FINETUNE = ['ABBA']

finetune_results = {}
tfrecord_info = {'ABBA': 100}

for artist in ARTISTS_TO_FINETUNE:
    tfrecord_path = os.path.join("./MIDI/Artist_MIDI/training_data", f'{artist}_melodies.tfrecord')
    
    if artist not in tfrecord_info:
        print(f"⚠ {artist} not in preprocessed data, skipping")
        continue
    
    if not os.path.exists(tfrecord_path):
        print(f"⚠ {artist} TFRecord not found, skipping")
        continue
    
    result = fine_tune_melody_rnn(
        artist,
        tfrecord_path,
        FINETUNE_CONFIG,
        MELODY_RNN_SAVE_PATH
    )
    
    finetune_results[artist] = result

print("\n" + "="*60)
print("FINE-TUNING SUMMARY")
print("="*60)

for artist, result in finetune_results.items():
    print(f"\n{artist}:")
    print(f"  Steps trained: {result['steps_trained']}")
    print(f"  Train directory: {result['train_dir']}")
    print(f"  Checkpoint: {result['final_checkpoint']}")


FINE-TUNING MODELS

🎹 Fine-tuning for ABBA...
  Learning rate: 0.0001
  Training steps: 5000
  Batch size: 32

  Starting training...



AttributeError: 'MelodyRnnModel' object has no attribute 'loss'

In [ ]:
## Step 4: Load Fine-Tuned Model for Generation

# Choose which model to use for generation
USE_FINETUNED = False  # Set to True to use fine-tuned model
FINETUNED_ARTIST = 'Billy_Joel'  # Which artist's model to use

if USE_FINETUNED and FINETUNED_ARTIST in finetune_results:
    # Use fine-tuned model
    result = finetune_results[FINETUNED_ARTIST]
    checkpoint_path = result['final_checkpoint']
    
    print(f"✓ Using fine-tuned model for {FINETUNED_ARTIST}")
    print(f"  Checkpoint: {checkpoint_path}")
    
    # Load fine-tuned bundle
    generation_bundle = bundle  # Use same bundle but with fine-tuned weights
    generation_model = melody_rnn_model.MelodyRnnModel(config)
    
    # Note: To fully use fine-tuned checkpoint, would need to construct
    # a generator with the checkpoint. For now, we'll use pre-trained.
    print("  Note: Generation will use pre-trained model for now")
    print("  Full checkpoint restoration requires custom setup")
else:
    # Use pre-trained model
    print("✓ Using pre-trained Melody_RNN model")
    generation_bundle = bundle
    generation_model = melody_rnn_model.MelodyRnnModel(config)

print(f"\nModel ready for generation")


In [44]:
# Create sequence generator from the loaded bundle
details = bundle.generator_details
model = melody_rnn_model.MelodyRnnModel(config)

sequence_generator = melody_rnn_sequence_generator.MelodyRnnSequenceGenerator(
    model=model,
    details=config.details,
    steps_per_quarter=config.steps_per_second,
    checkpoint=None,
    bundle=bundle,
)

print(f"\n🎹 Generating {NUM_GENERATIONS} melody candidates...\n")

generated_melodies = []
generated_sequences = []

for i in range(NUM_GENERATIONS):
    try:
        # Create a primer sequence with starting notes
        primer_notes = [60, 64, 67]  # C-E-G
        primer_sequence = note_seq.NoteSequence()
        
        current_time = 0
        note_duration = 0.5  # Quarter note
        
        for pitch in primer_notes:
            note = primer_sequence.notes.add()
            note.start_time = current_time
            note.end_time = current_time + note_duration
            note.pitch = pitch
            note.velocity = 80
            current_time += note_duration
        
        # Create GeneratorOptions
        generator_options = generator_pb2.GeneratorOptions()

        # Calculate generate time
        seconds_per_step = 1.0 / sequence_generator.steps_per_quarter
        generate_end_time = primer_sequence.total_time + (NUM_STEPS * seconds_per_step)

        # Set generation section (from end of primer, extending forward)
        generate_section = generator_options.generate_sections.add()
        generate_section.start_time = primer_sequence.total_time
        generate_section.end_time = generate_end_time

        # Set temperature
        generator_options.args['temperature'].float_value = TEMPERATURE
        generated_sequence = sequence_generator.generate(primer_sequence, generator_options)
        
        # Extract note sequence (skip primer notes, keep only generated notes)
        if generated_sequence and len(generated_sequence.notes) > len(primer_notes):
            notes = [int(note.pitch) for note in generated_sequence.notes[len(primer_notes):]]
            
            if len(notes) > 3:  # Only keep melodies with enough notes
                generated_melodies.append(notes)
                generated_sequences.append(generated_sequence)
                
                # Score this melody
                score = evaluator.evaluate(notes)
                print(f"  [{i+1:2d}] Generated melody ({len(notes)} notes) - Score: {score:.1f}/100")

    except Exception as e:
        if i == 0:  # Show detailed error on first failure
            print(f"  [{i+1:2d}] Generation failed: {type(e).__name__}: {str(e)}")
            import traceback
            traceback.print_exc()
        continue

print(f"\n✓ Generated {len(generated_melodies)} valid melodies")


🎹 Generating 20 melody candidates...

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from C:\Users\adamc\AppData\Local\Temp\tmp6etxwpy9\model.ckpt
  [ 1] Generation failed: SequenceGeneratorError: Got GenerateSection request for section that is before the end of the NoteSequence. This model can only extend sequences. Requested start time: 0.0, Final note end time: 1.5

✓ Generated 0 valid melodies


Traceback (most recent call last):
  File "C:\Users\adamc\AppData\Local\Temp\ipykernel_22408\1491662270.py", line 49, in <module>
    generated_sequence = sequence_generator.generate(primer_sequence, generator_options)
  File "c:\Users\adamc\miniconda3\envs\magenta_legacy\lib\site-packages\magenta\models\shared\sequence_generator.py", line 194, in generate
    return self._generate(input_sequence, generator_options)
  File "c:\Users\adamc\miniconda3\envs\magenta_legacy\lib\site-packages\magenta\models\melody_rnn\melody_rnn_sequence_generator.py", line 80, in _generate
    raise sequence_generator.SequenceGeneratorError(
magenta.models.shared.sequence_generator.SequenceGeneratorError: Got GenerateSection request for section that is before the end of the NoteSequence. This model can only extend sequences. Requested start time: 0.0, Final note end time: 1.5


## Score and Rank Generated Melodies

Evaluate all generated melodies and rank them by quality score.


In [ ]:
print("📊 Analyzing generated melodies...\n")

# Score all melodies
scores = []
for i, melody in enumerate(generated_melodies):
    score = evaluator.evaluate(melody)
    scores.append({
        'index': i,
        'score': score,
        'notes': melody,
        'length': len(melody),
        'unique_pitches': len(set(melody))
    })

# Sort by score (highest first)
scores_sorted = sorted(scores, key=lambda x: x['score'], reverse=True)

print("✓ Top Melodies by Quality Score:\n")
print(f"{'Rank':<6} {'Score':<8} {'Length':<8} {'Unique':<8} {'Range':<10}")
print("-" * 50)

for rank, entry in enumerate(scores_sorted[:10], 1):
    melody = entry['notes']
    pitch_range = max(melody) - min(melody) if melody else 0
    print(f"{rank:<6} {entry['score']:<8.1f} {entry['length']:<8} {entry['unique_pitches']:<8} {pitch_range:<10}")

print(f"\nAverage Score: {np.mean([s['score'] for s in scores]):.1f}/100")
print(f"Best Score: {scores_sorted[0]['score']:.1f}/100")
print(f"Worst Score: {scores_sorted[-1]['score']:.1f}/100")


## Save Top Melodies as MIDI

Export the best scoring melodies as playable MIDI files.


In [ ]:
import os

# Create output directory
output_dir = "./improved_melodies/"
os.makedirs(output_dir, exist_ok=True)

print("\n💾 Saving top melodies as MIDI files...\n")

# Save top 5 melodies
num_to_save = min(5, len(scores_sorted))

for rank, entry in enumerate(scores_sorted[:num_to_save], 1):
    melody_notes = entry['notes']
    score = entry['score']
    
    # Create a Music Sequence with timing
    sequence = note_seq.Sequence()
    sequence.tempo = 120  # BPM
    
    # Add notes with timing (quarter note = 0.5 beats at 120 BPM = 0.5 seconds)
    current_time = 0
    note_duration = 0.5  # Quarter note in seconds
    
    for pitch in melody_notes:
        note = sequence.notes.add()
        note.start_time = current_time
        note.end_time = current_time + note_duration
        note.pitch = int(pitch)
        note.velocity = 80
        current_time += note_duration
    
    # Save as MIDI
    filename = f"{output_dir}top_{rank}_score_{score:.0f}.mid"
    midi_io.note_sequence_to_midi_file(sequence, filename)
    print(f"  ✓ Saved: {filename} (Score: {score:.1f}/100)")

print(f"\n" + "="*60)
print("✓ Melody generation complete!")
print("="*60)
print(f"\nBest melodies saved to: {output_dir}")
print(f"\nTop 5 Scores:")
for rank, entry in enumerate(scores_sorted[:5], 1):
    print(f"  {rank}. Score: {entry['score']:.1f}/100 - {len(entry['notes'])} notes")


## Tune Music Theory Constraints

Adjust constraint weights to change melody generation characteristics.


In [ ]:
# Experiment with different constraint weights
print("\n📝 Tips for tuning music theory constraints:\n")

print("Current constraint_weights:")
for key, value in constraint_weights.items():
    print(f"  '{key}': {value}")

print("\nTo emphasize different characteristics, adjust weights:")
print("  - Increase 'smooth_intervals' for more stepwise motion")
print("  - Increase 'pitch_range' for more varied pitches")
print("  - Increase 'resolution' to prefer strong endings")
print("  - Decrease 'repetition' weight for more varied notes")

print("\nExample: More stepwise, fewer leaps")
print("  constraint_weights['smooth_intervals'] = 3.5  # Was 2.0")
print("  constraint_weights['pitch_range'] = 1.0       # Was 1.5")
print("  evaluator = MelodyEvaluator(constraint_weights)")

print("\nExample: More variety and uniqueness")
print("  constraint_weights['pitch_range'] = 2.5")
print("  constraint_weights['repetition'] = 0.3")
print("  evaluator = MelodyEvaluator(constraint_weights)")

print("\nThen re-run the generation cells to see how outputs change!")
